# CPSC 330 - Applied Machine Learning 

## Homework 3: Preprocessing 
### Associated lectures: [Lectures 4, 5, 6](https://ubc-cs.github.io/cpsc330/README.html) 

**Due date: Wednesday, Feb 01, 2023 at 11:59pm**

## Table of Contents

- [Instructions](#si)
- [Introduction](#in)
- [Exercise 1: Introducing the dataset](#1)
- [Exercise 2: Exploratory data analysis (EDA)](#2)
- [Exercise 3: Preprocessing](#3)
- [Exercise 4: Building models](#4)
- [Exercise 5: Evaluating on the test set](#5)

## Imports 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

ModuleNotFoundError: No module named 'sklearn'

## Instructions <a name="si"></a>
<hr>
rubric={points:6}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2022W2/blob/main/docs/homework_instructions.md). 

**You may work with a partner on this homework (maximum group size: 2).** 

_Note: The assignments will get gradually more open-ended as we progress through the course. In many cases, there won't be a single correct solution. Sometimes you will have to make your own choices and your own decisions (for example, on what parameter values to use when they are not explicitly provided in the instructions). Use your own judgment in such cases and justify your choices, if necessary._

## Introduction <a name="in"></a>
<hr>

A crucial step when using machine learning algorithms on real-world datasets is preprocessing. This homework will give you some practice of data preprocessing and building a supervised machine learning pipeline on a real-world dataset. 

<br><br>

## Exercise 1: Introducing the dataset <a name="1"></a>
<hr>

In this lab, you will be working on [the adult census dataset](https://www.kaggle.com/uciml/adult-census-income#). Download the CSV and save it as `adult.csv` locally in this homework folder. 

This is a classification dataset and the classification task is to predict whether income exceeds 50K per year or not based on the census data. You can find more information on the dataset and features [here](http://archive.ics.uci.edu/ml/datasets/Adult).

The starter code below loads the data CSV (assuming that it is saved as `adult.csv` in this folder). 

_Note that many popular datasets have sex as a feature where the possible values are male and female. This representation reflects how the data were collected and is not meant to imply that, for example, gender is binary._

In [ ]:
census_df = pd.read_csv("./adult.csv")
census_df.shape

<br><br>

### 1.1 Data splitting 
rubric={points:4}

In order to avoid violation of the golden rule, the first step before we do anything is splitting the data. 

**Your tasks:**

1. Split the data into `train_df` (60%) and `test_df` (40%) with `random_state = 42`. Keep the target column (`income`) in the splits so that we can use it in the exploratory data analysis.  

_Usually having more data for training is a good idea. But here we're using a 60%/40% split because this is kind of a big dataset for a modest laptop. A smaller training set means that it won't take too long to train the model on your laptop. A side advantage of this would be that with a bigger test split, we'll have a more reliable estimate of the deployment performance!_

In [ ]:
train_df, test_df = train_test_split(census_df, test_size=0.4, random_state=42)

<br><br>

## Exercise 2: Exploratory data analysis (EDA) <a name="2"></a> 
<hr>

Let's examine our `train_df`. 

In [ ]:
train_df.sort_index()

We see some missing values represented with a "?". Probably these were the questions not answered by some people during the census.  Usually `.describe()` or `.info()` methods would give you information on missing values. But here, they won't pick "?" as missing values as they are encoded as strings instead of an actual NaN in Python. So let's replace them with `np.nan` before we carry out EDA. If you do not do it, you'll encounter an error later on when you try to pass this data to a classifier. 

In [ ]:
train_df_nan = train_df.replace("?", np.nan)
test_df_nan = test_df.replace("?", np.nan)
train_df_nan.shape

In [ ]:
train_df_nan.sort_index()

The "?" symbols are now replaced with NaN values. 

<br><br>

### 2.1 Visualizing features
rubric={points:10}

**Your tasks:**

1. Examine the information given by `train_df_nan.info()` and `train_df_nan.describe()` methods. In case of `.describe()`, use the `include="all"` argument to show summary statistics of all  features.
2. Visualize the histograms of numeric features. 
3. From the visualizations, which features seem relevant for the given prediction task?

> Note: (Optional) If you're feeling excited about this you are welcome to use [`pandas_profiling`](https://github.com/pandas-profiling/pandas-profiling) for more elaborate visualization and EDA. 

In [ ]:
train_df_nan.info()

In [ ]:
train_df_nan.describe(include="all")

In [ ]:
train_df_nan.hist(bins=20, figsize=(20, 15))

Looking at the distributions of the histograms, we think that most of the numeric features seem relevant because the distributions are pretty diverse.
However, looking at `capital.gain` and `capital.loss`, the distributions are stacked in one column (not diverse) which suggests that the data may not
be useful in the task of predicting income. Furthermore, after further investigation by looking at the details of the dataset `adult.csv`, the data seems to be contaminated 
with many cells filled with values of `0` and `99999` (particularly in the column of `capital.gain`).

<br><br>

### 2.2 Identify transformations to apply
rubric={points:18}

**Your tasks:**
1. Identify the sequence of transformations that you would apply on each column in the dataset and fill in the table below accordingly. An example of the sequence of transformations to be applied on the `occupation` feature is shown in the table below. You may decide not to apply any transformations on a certain column or entirely drop a column from your model. That's totally fine. 
2. Are there common transformations you would like to apply on certain types of features? Identify different feature types for applying different transformations. In particular, fill in the lists below. 
3. Is including the `race` feature for predicting income ethically a good idea? Briefly discuss. 

> Note: This question is a bit open-ended and there is no single correct solution.

| Feature | Transformation |
| --- | ----------- |
| occupation | imputation, OHE |
| age | scaling |
| workclass | imputation, OHE |
| fnlwgt | scaling |
| education | ordinal encoding |
| education.num | scaling |
| marital.status | OHE |
| relationship | (drop) |
| race | OHE |
| sex | binary encoding |
| capital.gain | (drop) |
| capital.loss | (drop) |
| hours.per.week | scaling |
| native.country | imputation, OHE |


For numeric features, we would like to apply scaling by using `StandardScaler`. For categorical features with some null features, we would like to apply imputation by using `SimpleImputer`. For all categorical features, we would like to apply `OneHotEncoder`. For ordinal features, we would like to apply `OrdinalEncoder`. 

In [ ]:
# Fill in the lists below. 
# It's OK to keep some of the lists empty or add new lists. 
numeric_features = ["age", "fnlwgt", "education.num", "hours.per.week"]
nan_features = ["occupation", "workclass", "native.country"]
categorical_features = ["marital.status", "race"]
ordinal_features = ["education"]
binary_features = ["sex"]
drop_features = ["relationship", "capital.gain", "capital.loss"]
passthrough_features = []
target = "income"

It is a debatable issue of whether to include `race` in training our model for prediction. In the modern day and in the public view, including `race` for predicting income would not be considered to be ethical due to possible ensuing racism. However, we believe that, objectively, including race would be a good feature nonetheless due to work habits or other qualitative properties that are derived from the particular race's culture.

<br><br>

### 2.3 Separating feature vectors and targets  
rubric={points:4}

**Your tasks:**

1. Create `X_train`, `y_train`, `X_test`, `y_test` from `train_df_nan` and `test_df_nan`. 
2. At this point, if you train [`sklearn`'s `SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) model on `X_train` and `y_train` would it work? Why or why not?

In [ ]:
X_train = train_df_nan.drop(columns=[target])
y_train = train_df_nan[target]
X_test = test_df_nan.drop(columns=[target])
y_test = test_df_nan[target]

Up to this point, the model would not work. The `SVC` model only takes numerical inputs, while we have categorical features. These features are not yet transformed into numerical values to fit into the model. 

<br><br><br><br>

## Exercise 3: Preprocessing <a name="3"></a>
<hr>

### 3.1 Preprocessing using `sklearn`'s `ColumnTransformer` and `Pipeline`
rubric={points:18}

Let's carry out preprocessing using `sklearn`'s `ColumnTransformer` and `Pipeline`. Note that you can define pipelines in two ways: 
- by using [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) and explicitly providing named steps
- by using [`make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn.pipeline.make_pipeline), which automatically names the steps in the pipeline with their class names. 

Similarly you can create a column transformer in two ways:
- by using [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
- by using [`make_column_transformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_transformer.html) 

You may use the method of your choice but `make_pipeline` and `make_column_transformer` are highly recommended.  

**Your tasks:**

1. Create a column transformer `preprocessor` based on transformations you want to apply on the data from 2.2. 
2. Transform the data by calling `fit_transform` on the training set. What's the shape of the transformed data? 
3. Why do we need to use a column transformer in this case? Briefly explain. 

Before creating the `ColumnTransformer`, we need to consider the ordinal feature `education`. A glimpse of the data categories are as below: 

In [ ]:
edu_categories = train_df["education"].unique().tolist()
edu_categories

According to preprocessing work done by researchers (Reference: https://sites.google.com/site/complexdataminingproject/), a reasonable choice of ranking the features would be as follows: 

| Order | Category |
| --- | ----------- |
| 0 | Preschool |
| 1 | 1st-4th |
| 2 | 5th-6th |
| 3 | 7th-8th |
| 4 | 9th |
| 5 | 10th |
| 6 | 11th |
| 7 | 12th |
| 8 | HS-grad |
| 9 | Prof-school |
| 10 | Assoc-acdm |
| 11 | Assoc-voc |
| 12 | Some-college |
| 13 | Bachelors |
| 14 | Masters |
| 15 | Doctorate |

In [ ]:
edu_order = [["Preschool", "1st-4th", "5th-6th", "7th-8th", "9th", "10th", "11th", "12th", "HS-grad", 
              "Prof-school", "Assoc-acdm", "Assoc-voc", "Some-college", "Bachelors", "Masters", "Doctorate"]]

We then construct the `ColumnTransformer` based on the order above. 

In [ ]:
preprocessor = make_column_transformer(
    (
        make_pipeline(SimpleImputer(strategy="most_frequent"), 
                     OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False)), 
                     nan_features
    ), 
    (StandardScaler(), numeric_features), 
    (OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False), categorical_features + binary_features), 
    (OrdinalEncoder(categories=edu_order), ordinal_features), 
    ("passthrough", passthrough_features), 
    ("drop", drop_features)
)
preprocessor

In [ ]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_train_preprocessed

In [ ]:
X_train_shape = pd.DataFrame(X_train_preprocessed).shape
X_train_shape

In [ ]:
pd.DataFrame(X_train_preprocessed)

The shape of the transformed data is 19536 rows by 81 columns, meaning that the dataset has 19536 entries and each entries contains 81 features. 

We need to create a `ColumnTransformer` because we have different transformations to be applied into different features. For example, `OneHotEncoder` is only applied to categorical features but we do not want to apply onto numerical features. Therefore, we need to use a `ColumnTransformer` so that various features can have their own unique data transformations. 

<br><br><br><br>

## Exercise 4: Building models <a name="4"></a>
<hr>

Now that we have preprocessed features, we are ready to build models. Below is the function we used in class, which returns the mean cross-validation score along with standard deviation for a given model. Feel free to use it to keep track of your results if you like. 

In [ ]:
results_dict = {} # dictionary to store all the results

In [ ]:

def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

<br><br>

### 4.1 Baseline model 
rubric={points:6}

**Your tasks:**
1. Define a pipeline with two steps: `preprocessor` from 3.1 and `scikit-learn`'s `DummyClassifier` with `strategy="prior"` as your classifier.  
2. Carry out 5-fold cross-validation with the pipeline. Store the results in `results_dict` above. Display the results as a pandas DataFrame.  

> You may use the function `mean_std_cross_val_scores` above to carry out cross-validation and storing results. Refer to the class notes if you are unsure about how to use it. 

In [ ]:
pipe_dummy = make_pipeline(preprocessor, DummyClassifier(strategy="prior"))

In [ ]:
results_dict["Dummy Classifier"] = mean_std_cross_val_scores(pipe_dummy, X_train, y_train, cv=5, return_train_score=True)

In [ ]:
pd.DataFrame(results_dict)

<br><br>

### 4.2 Trying different classifiers
rubric={points:14}

**Your tasks:**

1. For each of the models in the starter code below: 
    - Define a pipeline with two steps: `preprocessor` from 3.1 and the model as your classifier. 
    - Carry out 5-fold cross-validation with the pipeline.  
    - Store the results in `results_dict`. 
2. Display all the results so far as a pandas dataframe. 
3. Compare the train and validation accuracies and `fit` and `score` times in each case. How do the the validation accuracies compare to the baseline model from 4.1? Which model has the best validation accuracy? Which model is the fastest one?  

> Note that this might take a while to run.

> You may use the function above `mean_std_cross_val_scores` to carry out cross-validation and storing results. Refer to the class notes if you are unsure about how to use it. 

In [ ]:
models = {
    "decision tree": DecisionTreeClassifier(),
    "kNN": KNeighborsClassifier(),
    "RBF SVM": SVC(),
}

In [ ]:
for model_name, model in tqdm(models.items()):
    pipe = make_pipeline(preprocessor, model)
    results_dict[model_name] = mean_std_cross_val_scores(pipe, X_train, y_train, cv=5, return_train_score=True, error_score='raise')

In [ ]:
pd.DataFrame(results_dict)

The `DummyClassifier` performs the worst in the training and validation score. It is expected because the `DummyClassifier` acts as the baseline model. 

For the training scores, the `DecisionTree` classifier has the highest score, followed by `kNN` and `SVC`. It is notable that the `DecisionTree` classifier is overfitting to the data. 

For the validation scores/accuracies, the `SVM` with `RBF`-kernel classifier has the highest score, followed by `kNN` and `DecisionTree`. 

* The `SVM` with `RBF`-kernel classifier has the highest validation accuracy. 

* Other than the `DummyClassifier`, the `DecisionTree` is the fastest one. 

<br><br>

### (optional) 4.3 Exploring the importance of scaling
rubric={points:1}

In this exercise you'll examine whether scaling helps in case of KNNs and SVM RBFs. 

**Your tasks:**

1. Create a column transformer without the `StandardScaler` step for `numeric_features`. 
2. Repeat the steps in 4.2 with this new column transformer. 
3. Compare the results of scaled numeric features with unscaled numeric features. Is scaling necessary for decision trees? Why or why not?

In [ ]:
preprocessor2 = make_column_transformer(
    (
        make_pipeline(SimpleImputer(strategy="most_frequent"), 
                     OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False)), 
                     nan_features
    ), 
    # StandardScaler for numeric features is not included in this column transformer
    # (StandardScaler(), numeric_features), 
    (OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False), categorical_features + binary_features), 
    (OrdinalEncoder(categories=edu_order), ordinal_features), 
    ("passthrough", passthrough_features), 
    ("drop", drop_features)
)
preprocessor2

In [ ]:
models = {
    "kNN": KNeighborsClassifier(),
    "RBF SVM": SVC(),
}

In [ ]:
for model_name, model in tqdm(models.items()):
    pipe = make_pipeline(preprocessor2, model)
    results_dict[model_name + " (no scaled numeric feats)"] = mean_std_cross_val_scores(pipe, X_train, y_train, cv=5, return_train_score=True, error_score='raise')

In [ ]:
pd.DataFrame(results_dict)

By considering both `kNN` and `SVM` with `RBF`-kernel classifier, both the `fit` and `score` time have slightly increased since the numeric features are not scaled. It takes greater computational power for the computer to calculate larger numbers. 

In general, both the training and validation scores for both models have slightly decreased when the numeric features are scaled, except that the validation score for `kNN` has slightly improved. 

Scaling numeric features is not necessary for decision trees. As the algorithm of the `DecisionTreeClassifier` suggests, the model will iterate across different threshold values and picks the best combination which maximizes the information gain (equivalently, minimizes the entropy). Scaling has no effect on picking the optimal values as the number of threshold combinations to be trained are the same, before and after scaling. 

### 4.4 Hyperparameter optimization
rubric={points:10}

In this exercise, you'll carry out hyperparameter optimization for the hyperparameter `C` of SVC RBF classifier. In practice you'll carry out hyperparameter optimization for all different hyperparameters for the most promising classifiers. For the purpose of this assignment, we'll only do it for the `SVC` classifier with one hyperparameter: `C`. 

**Your tasks:**

1. For each `C` value in the `param_grid` in the starter code below: 
    - Create a pipeline object with two steps: preprocessor from 3.1 and `SVC` classifier with the value of `C`.
    - Carry out 5-fold cross validation with the pipeline.  
    - Store the results in `results_dict` and display results as a pandas DataFrame. 
2. Which hyperparameter value seems to be performing the best, and why? Is it different than the default value for the hyperparameter used by `scikit-learn`? 

> Note: Running this might take a while. 

In [ ]:
param_vals = np.logspace(-2, 2, num=5, base=10)
param_grid = {"C": np.delete(param_vals, np.where(param_vals == 1.0))}
param_grid

In [ ]:
c_values = param_grid["C"]
for i in tqdm(range(len(c_values))):
    c = c_values[i]
    pipe = make_pipeline(preprocessor, SVC(C=c))
    results_dict[f"RBF SVM (C={c})"] = mean_std_cross_val_scores(pipe, X_train, y_train, cv=5, return_train_score=True, error_score='raise')

In [ ]:
pd.DataFrame(results_dict)

From the results above, we can see that the SVC RBF classifier performs the best when `C=100` although the model slightly overfits compared to `C=0.1` or `C=10`. This value of `C` is the highest among the `param_grid`, meaning it has the most complex structure. A model with a more complicated structure can capture the edge cases for classification, therefore it seems to result in a higher training score and validation score. 

The value `C=100` is different from the default value of `C` used by `scikit-learn`, which is `1`. 

<br><br><br><br>

## Exercise 5: Evaluating on the test set <a name="5"></a>
<hr>

Now that we have a best performing model, it's time to assess our model on the set aside test set. In this exercise you'll examine whether the results you obtained using cross-validation on the train set are consistent with the results on the test set. 

### 5.1 Scoring on the unseen test set 
rubric={points:10}

**Your tasks:**

1. Train the best performing model on the entire training set. 
2. Report the results of this model on `X_test`. 
3. Are the cross-validation results and test results consistent? 

In [ ]:
# train the model
best_model = SVC(C=100)
pipe = make_pipeline(preprocessor, best_model)
pipe.fit(X_train, y_train)

In [ ]:
# test the model
pipe.predict(X_test)
test_score = pipe.score(X_test, y_test)
round(test_score, 3)

The cross-validation results `(accuracy: 0.839)` and the test results `(accuracy: 0.834)` are consistent, because our model is at the "sweet spot" where the approximation error is minimized. The validation score gives a consistent estimate of the test score. 

## Exercise 6: Summary

rubric={Pass/Fail}

You are all done with Homework 3! Your last task is reflecting on what you have learned and answering the following questions. **This task is necessary to pass HW3**. Failing to complete this task means you will not receive points for this assignment.

1. In about 100 words, describe what you have learned from this homework.
2. Write at least one well-formulated question on something that is still not clear about the content of this homework, or you would like to know more about.

1. From this assignment we have learned to recognize when to use `ColumnTransformer` and `Pipeline` during preprocessing, thus differentiating between them based on their use cases. <br> I (Adrian) had done similar machine learning tasks when I was at a tech startup internship, however, I have not used the best practice to preprocess the data. Therefore, this assignment gave me insight into preprocessing data in an organized fashion. Furthermore, this assignment also reinforced our knowledge of how to apply column transformers and pipelines to a dataset using scikit learn (among brushing up on using pandas to display observations). <br> I (Oliver) did not know that pipelines could be used within column transformers (and not very familiar with the python programming language). This includes reading up on the documentation on the scikit learn website to determine the specific parameter names for the various models like SVC.

2. Questions in mind: 
* Why `GridSearchCV` and `RandomizedSearchCV` is not introduced in this lecture (or at this stage) when we are supposed to learn hyperparameter optimization?
* From this assignment, we used 5-fold validation to use with the pipelines. Why did we specifically choose `cv=5` instead of exploring a range of values to determine the best one?
* Can you send me the notebook after you apply these changes? :thumbsup:

<br><br><br><br>

## Submission instructions 

**PLEASE READ:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 

This was a tricky one but I hope you are feeling good after working on it. You are now ready to build a simple supervised machine learning pipeline on real-world datasets! Well done (**and don't forget to submit**)! 

![](./eva-well-done.png)

